# Bloonet Jupyter Notebook

## Required imports

In [ ]:
import requests
import uuid
import json
import time
import shutil
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display
from typing import Union
from pathlib import Path

## Required constants
Update your authentication information before running a calculation.

In [ ]:
SESSION_UUID = uuid.uuid4()
SIMULATION_UUID = uuid.uuid4()
BWS_API_URL = "https://api.bloonetws.siradel.com"
BWS_DL_URL = "https://dl.bloonetws.siradel.com"
RECEPTION_HEIGHTS = [2]
RESOLUTION = 10
ACCESS_TOKEN = None
REFRESH_TOKEN = None
AUTHENTICATION = {
    "url": "https://keycloak.bloonetws.siradel.com/realms/volcanoweb/protocol/openid-connect/token",
    "clientId": "volcano-web-cli",
    "username": "USERNAME",
    "password": "PASSWORD"
}

## Required functions

In [ ]:
def call_request(method: str, url: str, authentication_data: dict,
                 files: list = None, json_content: Union[dict, list] = None,
                 params: dict = None, retry: int = 0, timeout=None) -> requests.Response:
    """
    Call requests with args for use authenticate

    @param method: {str} method request
    @param url: {str} url request
    @param authentication_data: {dict} authentication data
    @param files: {list} files request
    @param json_content: {dict} json request
    @param params: {dict} params request
    @param retry: {int} number of retry
    @param timeout: {Any} timeout request

    @return: {request.Response} response of request
    """

    headers = None

    if authentication_data is None:
        print("Authentication data needed")
    token = get_access_token(authentication_data)
    headers = {"Authorization": f"Bearer {token}"}

    res = None
    if method.upper() == "POST":
        res = requests.post(url=url, files=files, json=json_content, params=params,
                            timeout=timeout, headers=headers)
    elif method.upper() == "PUT":
        res = requests.put(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers)
    elif method.upper() == "GET":
        res = requests.get(url=url, files=files, json=json_content, params=params,
                           timeout=timeout, headers=headers)
    elif method.upper() == "DELETE":
        res = requests.delete(url=url, files=files, json=json_content, params=params,
                              timeout=timeout, headers=headers)
    else:
        raise ValueError("Method name %S not implemented", method.upper())

    # If access_token expires, refresh the token one time and recall requests
    if res.status_code == 403 and retry == 0:
        refresh_token(authentication_data)
        res = call_request(
            method,
            url,
            authentication_data,
            files,
            json_content,
            params,
            1,
            timeout
        )

    return res


def get_access_token(authentication_data: dict) -> str:
    """
    Retrieve an access token

    @param authentication_data: {dict} authentication data

    @return: {str} access token provided
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if ACCESS_TOKEN is not None:
        return ACCESS_TOKEN
    payload = {
        "grant_type": "password",
        "client_id": authentication_data["clientId"],
        "username": authentication_data["username"],
        "password": authentication_data["password"]
    }

    response = requests.post(
        authentication_data["url"],
        data=payload
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get access token: %s", json_response["error_description"])
    return ACCESS_TOKEN


def refresh_token(authentication_data: dict) -> None:
    """
    Refresh the access token

    @param authentication_data: {dict} authentication data
    """

    global ACCESS_TOKEN, REFRESH_TOKEN
    if REFRESH_TOKEN is None:
        raise RuntimeError("Error call refresh token : refresh token is empty")

    payload = {
        "grant_type": "refresh_token",
        "client_id": authentication_data["clientId"],
        "refresh_token": REFRESH_TOKEN
    }

    response = requests.post(
        authentication_data["url"],
        data=payload
    )

    json_response = response.json()

    if response.status_code == 200:
        ACCESS_TOKEN = json_response["access_token"]
        REFRESH_TOKEN = json_response["refresh_token"]
    else:
        raise RuntimeError("Error call get refresh token: %s", json_response["error_description"])


def is_same_base_station(base_station_1: dict, base_station_2: dict) -> bool:
    """
    Check if the two base stations in parameter are the same

    @param base_station_1: {dict} first base station to compare
    @param base_station_2: {dict} second base station to compare

    @return: {bool} True if equals, False else
    """
    return (
            base_station_1["networkId"] == base_station_2["networkId"] and
            base_station_1["name"] == base_station_2["name"] and
            base_station_1["x"] == base_station_2["x"] and
            base_station_1["y"] == base_station_2["y"] and
            base_station_1["z"] == base_station_2["z"] and
            base_station_1["azimuth"] == base_station_2["azimuth"] and
            base_station_1["downtilt"] == base_station_2["downtilt"] and
            base_station_1["carrierFrequency"] == base_station_2["carrierFrequency"] and
            base_station_1["transmitPower"] == base_station_2["transmitPower"]
    )


def extract_rows_from_json(obj: dict, metric_name: str) -> list: 
    """
    Extract transmitter-receiver metric rows from a JSON object.

    @param obj: {dict} JSON-like object representing simulation results, expected to contain
        a "transmitters" key with nested receiver data.
    @param metric_name: {str} Name of the metric to extract from receiver qualifications
        (e.g., "received_power").

    @return: {list} A list of dictionaries, each with keys:
        - "receiver": Receiver name
        - "transmitter": Transmitter name
        - <metric_name>: Extracted metric value (or None if not found)
    """
    rows = []
    for tx in obj.get("transmitters", []):
        tx_name = tx.get("name")
        for rx in tx.get("receivers", []):
            rx_name = rx.get("name")
            value = None
            for q in rx.get("qualifications", []):
                if q.get("name") == metric_name:
                    value = q.get("value")
                    break
            rows.append({"receiver": rx_name, "transmitter": tx_name, metric_name: value})
    return rows


def get_public_models() -> list:
    """
    Retrieve the public models

    @return: {list} list of public models
    """
    response = call_request(
        method="GET",
        url=f"{BWS_API_URL}/propagationmodels",
        authentication_data=AUTHENTICATION
    )
    public_models_list = response.json()

    if response.status_code != 200:
        raise RuntimeError("Error call get public models: %s", public_models_list["error_description"])

    PUBLIC_MODELS_NAME = ["fixed wireless access", "mobility"]
    public_models = [model for model in public_models_list if model["name"].lower() in PUBLIC_MODELS_NAME and model["type"] is None]
    return public_models


def get_public_antennas() -> list:
    """
    Retrieve the public antennas

    @return: {list} list of public antennas
    """
    response = call_request(
        method="GET",
        url=f"{BWS_API_URL}/antennas",
        authentication_data=AUTHENTICATION
    )
    public_antennas_list = response.json()

    if response.status_code != 200:
        raise RuntimeError("Error call get public antennas: %s", public_antennas_list["error_description"])

    PUBLIC_ANTENNAS_NAME = [
        "tarana-bn-3ghz-compact-r0",
        "tarana-bn-5ghz-r1",
        "tarana-bn-6ghz-r2",
        "tarana-bn-6ghz-x2-r2"
    ]
    public_antennas = [antenna for antenna in public_antennas_list if antenna["name"].lower() in PUBLIC_ANTENNAS_NAME]
    return public_antennas


## UI elements

In [ ]:
# Widgets upload
label_tx = widgets.Label("Select the CSV file for the transmitters:")
upload_tx = widgets.FileUpload(accept=".csv", multiple=False)

label_rx = widgets.Label("Select the CSV file for the receivers:")
upload_rx = widgets.FileUpload(accept=".csv", multiple=False)

# Widgets dropdown
label_model = widgets.Label("Select the propagation model to use:")
dropdown_model = widgets.Dropdown(
    options=[(m["name"], m["uuid"]) for m in get_public_models()]
)

label_antenna = widgets.Label("Select the antenna diagram to use:")
dropdown_antenna = widgets.Dropdown(
    options=[(m["name"], m["uuid"]) for m in get_public_antennas()]
)

# Display areas
out_tx = widgets.Output()
out_rx = widgets.Output()
out_model = widgets.Output()
out_antenna = widgets.Output()
out_status = widgets.Output()

# Button to start the calculation
btn_run = widgets.Button(
    description="Start the calculation",
    button_style="primary",
    disabled=True
)

# Layout
ui = widgets.VBox([
    label_tx, upload_tx, out_tx,
    widgets.HTML("<hr>"),
    label_rx, upload_rx, out_rx,
    widgets.HTML("<hr>"),
    label_model, dropdown_model, out_model,
    widgets.HTML("<hr>"),
    label_antenna, dropdown_antenna, out_antenna,
    widgets.HTML("<hr>"),
    btn_run,
    out_status
])

# Global variables for storing DataFrames
df_tx = None
df_rx = None


## UI callback functions

In [ ]:
def handle_upload_tx(change: dict) -> None:
    """
    Handle the upload of transmitter files and update the global DataFrame.

    @param change: {dict} The change event triggered by the file upload widget.
    @raises Exception: If reading the CSV file fails, an error message is displayed and df_tx is set to None.
    """
    global df_tx
    out_tx.clear_output()

    if not upload_tx.value:
        with out_tx:
            print("No transmitters files")
        df_tx = None
        update_button_state()
        return

    # Content recovery
    files = upload_tx.value
    file_info = files[0]
    filename = file_info["name"]
    content = file_info["content"]

    try:
        df_tx = pd.read_csv(io.BytesIO(content), sep=";")
        with out_tx:
            print(f"Transmitters file loaded: {filename}")
            display(df_tx.head())
    except Exception as e:
        df_tx = None
        with out_tx:
            print(f"Error reading CSV transmitters: {e}")

    update_button_state()


def handle_upload_rx(change: dict) -> None:
    """
    Handle the upload of receiver files and update the global DataFrame.

    @param change: {dict} The change event triggered by the file upload widget.
    @raises Exception: If reading the CSV file fails, an error message is displayed and df_rx is set to None.
    """
    global df_rx
    out_rx.clear_output()

    if not upload_rx.value:
        with out_rx:
            print("No receivers file")
        df_rx = None
        update_button_state()
        return

    files = upload_rx.value
    file_info = files[0]
    filename = file_info["name"]
    content = file_info["content"]

    try:
        df_rx = pd.read_csv(io.BytesIO(content), sep=";")
        with out_rx:
            print(f"Receivers file loaded: {filename}")
            display(df_rx.head())
    except Exception as e:
        df_rx = None
        with out_rx:
            print(f"Error reading CSV receivers: {e}")

    update_button_state()


def update_button_state() -> None:
    """
    Update the enabled/disabled state of the run button based on input validity.

    @summary:
        This function checks whether all required inputs (transmitter DataFrame, receiver DataFrame,
        selected model, and selected antenna) are available and valid. If any of these conditions
        are not met, the run button is disabled; otherwise, it is enabled.
    """
    btn_run.disabled = not (
        isinstance(df_tx, pd.DataFrame) and 
        isinstance(df_rx, pd.DataFrame) and 
        dropdown_model.value and 
        dropdown_antenna.value
    )


# Connect callbacks to widgets
upload_tx.observe(handle_upload_tx, names="value")
upload_rx.observe(handle_upload_rx, names="value")


## API calls functions

In [ ]:
def create_session() -> dict:
    """
    Create a new session on the BWS server and return its details.

    @return: {dict} A dictionary containing the session information returned by the BWS API.
    @raises RuntimeError: If the BWS API does not return a 201 status code, indicating session creation failed.
    """

    # Session information
    session = {
        "uuid": str(SESSION_UUID),
        "name": "session_notebook_mp2mp",
        "description": "Session sample MP to MP from Jupyter Notebook"
    }

    # Post session to BWS server
    response = call_request(
        method="POST",
        url=f"{BWS_API_URL}/sessions",
        authentication_data=AUTHENTICATION,
        json_content=session
    )
    result = response.json()

    if response.status_code != 201:
        raise RuntimeError("Error call create session: %s", result["error_description"])
    
    return result


def create_simulation(network_list: pd.DataFrame, antenna_uuid: str, model_uuid: str) -> list:
    """
    Build and submit a multi-point to multi-point simulation request to the BWS server.

    @param network_list: {pd.DataFrame} Structured table of links (transmitters/receivers) and radio parameters.
    @param antenna_uuid: {str} UUID of the antenna to associate with each base station.
    @param model_uuid: {str} UUID of the propagation model to apply.

    @return: {list} The BWS API response body (parsed JSON) describing the created simulation.
    @raises RuntimeError: If the BWS API does not return HTTP 201 (simulation creation failed).

    """
    # Create simulation object
    propagation_list = []
    for _, network in network_list.iterrows():

        new_base_station = {
            "x": float(network["transmitter longitude"]),
            "y": float(network["transmitter latitude"]),
            "z": float(network["transmitter height"]),
            "epsgCode": 4326,
            "zmeaning": "ZMEANING_GROUND",
            "azimuth": float(network["azimuth"]),
            "downtilt": float(network["downtilt"]),
            "carrierFrequency": float(network["frequency"]),
            "description": "",
            "sessionUuid": str(SESSION_UUID),
            "name": network["transmitter name"],
            "networkId": network["transmitter id"],
            "transmitPower": float(network["emitting power"]),
            "antennaUuid": antenna_uuid
        }

        new_user_equipment = {
            "sessionUuid": str(SESSION_UUID),
            "description": "",
            "zmeaning": "ZMEANING_GROUND",
            "type": "POINT",
            "name": network["receiver name"],
            "heights": [network["receiver height"]],
            "coordinates": {
                "x": network["receiver longitude"],
                "y": network["receiver latitude"],
                "epsgCode": 4326
            }
        }

        already_exist = False
        for propagation in propagation_list:
            base_station = propagation["baseStation"]
            if is_same_base_station(base_station, new_base_station):
                if new_user_equipment["type"] == "POINT":
                    propagation["userEquipments"].append(new_user_equipment)
                    already_exist = True
                    break

        if not already_exist:
            new_propagation = {
                "baseStation": new_base_station,
                "userEquipments": [new_user_equipment],
                "propagationModelUuid": model_uuid
            }
            propagation_list.append(new_propagation)
        
    simulation_request = {
        "uuid": str(SIMULATION_UUID),
        "name": "session_sample_mp2mp_notebook_simulation",
        "calculationSessionUuid": str(SESSION_UUID),
        "propagationRequest": {
            "propagationScenarios": propagation_list,
            "resultTypes": ["RECEIVED_POWER"],
            "heights": RECEPTION_HEIGHTS,
            "zmeaning": "ZMEANING_GROUND"
        }
    }

    multipart_form_data = [("json", (None, json.dumps(simulation_request), "application/json")), *[]]

    # Post simulation to BWS server
    response = call_request(
        method="POST",
        url=f"{BWS_API_URL}/simulations",
        authentication_data=AUTHENTICATION,
        files=multipart_form_data
    )
    simulation = response.json()

    if response.status_code != 201:
        raise RuntimeError("Error call create session: %s", simulation["error_description"])
    
    return simulation


def wait_for_simulation():
    """
    Poll the BWS API for the current simulation status until it finishes or errors.

    @summary:
        Periodically queries the simulation status endpoint every 5 seconds, prints the
        reported progress to the `out_status` output widget, and stops when the simulation
        reaches state "DONE" or "ERROR". In case of an error state, it also prints the error
        message and error details.
    """

    while True:
        time.sleep(5)
        response = call_request(
            method="GET",
            url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/status",
            authentication_data=AUTHENTICATION
        ).json()
        
        with out_status:
            print(f"Progress: {response['progress']}%")
            
            if response["state"] == "DONE":
                print(f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']}")
                break
            if response["state"] == "ERROR":
                print(f"Simulation {str(SIMULATION_UUID)} finished with state {response['state']}")
                print(f"with message {response['error']}")
                print(response['errorMessages'])
                break


def download_simulation_results() -> list:
    """
    Download all generated result files for the current simulation and return their local paths.

    @return: {list} A list of dictionaries with normalized file names and absolute file paths.
    """

    simulation_results = []
    # Create and/or clean the tmp results folder
    results_path = Path("/tmp/results")
    if results_path.exists():
        shutil.rmtree(results_path, ignore_errors=True)
    results_path.mkdir(parents=True, exist_ok=True)
    # Retrieve the simulation results
    res = call_request(
        method="GET",
        url=f"{BWS_API_URL}/simulations/{str(SIMULATION_UUID)}/results",
        authentication_data=AUTHENTICATION
    )

    results = res.json()

    with out_status:
        if res.status_code in (404, 400):
            print(f"Error simulation {SIMULATION_UUID}: {results['message']}")
            return
        for result in results:
            # Download the tif files
            current_tiff = call_request(
                method="GET",
                url=f"{BWS_DL_URL}/results/{str(result['uuid'])}/download",
                authentication_data=AUTHENTICATION
            )
            
            if current_tiff.status_code != 200:
                print(f"Error downloading {result['fileName']}")
            
            # Save this tif files in tmp folder
            file_path = f"/tmp/results/{result['fileName']}"
            with open(file_path, "wb") as result_file:
                result_file.write(current_tiff.content)
            
            filename = result["fileName"].rsplit(".", 1)[0]
            
            simulation_results.append({
                "name": filename.replace("-", "_").replace(".", "_"),
                "path": file_path
            })
    
    return simulation_results


def generate_mp2mp_matrix(results_file: list, metric_name="received_power"):
    """
    Build a receiver × transmitter matrix DataFrame from MP-to-MP JSON result files.

    @summary:
        Iterates over a list of result file descriptors (each containing a local file path),
        loads the JSON payload, extracts long-form rows using `extract_rows_from_json()`,
        and pivots them into a wide-form matrix:
            - index: receiver names
            - columns: "<transmitter>_<metric_name>"
            - values: metric values (e.g., received power)
        Returns the resulting `pandas.DataFrame`. If no rows are found, returns an empty DataFrame.

    @param results_file: {list} List of result file descriptors. Each item must include a `"path"` key
        pointing to a local JSON file produced by the MP-to-MP process.
    @param metric_name: {str} Metric to extract and pivot (default: "received_power").

    @return: {pd.DataFrame} Wide-form matrix with receivers in rows and transmitters as metric-specific columns.
    @raises FileNotFoundError: If any referenced result file path does not exist.
    @raises RuntimeError: If a result file cannot be read or parsed as JSON.
    """

    all_rows = []
    for result_file in results_file:
        result_file_path = Path(result_file["path"])
        if not result_file_path.exists():
            raise FileNotFoundError(f"File {result_file_path} not found")
        try:
            with open(result_file_path, "r", encoding="utf-8") as file:
                data = json.load(file)
            all_rows.extend(extract_rows_from_json(data, metric_name=metric_name))
        except Exception as e:
            raise RuntimeError(f"Cannot read {result_file_path} file: {e}")
    df_long = pd.DataFrame(all_rows)
    if df_long.empty:
        return pd.DataFrame()
    
    df = df_long.pivot_table(
        index="receiver",
        columns="transmitter",
        values=metric_name,
        aggfunc="first"
    )
    df.columns = [f"{c}_{metric_name}" for c in df.columns]
    df = df.sort_index()
    return df


## Calculation actions

In [ ]:
def on_run_clicked(b: widgets.Button) -> None:
    """
    Executes the simulation workflow when the button is clicked in the UI.

    This function:
      1. Validates that both datasets `df_tx` (transmitters) and `df_rx` (receivers) are loaded,
         and that a propagation model (`dropdown_model`) and an antenna (`dropdown_antenna`) are selected.
      2. Builds the Cartesian product TXxRX to create the network and assigns a unique,
         incremental transmitter ID ("transmitter id").
      3. Creates a session, initializes and runs the simulation, waits for completion,
         then downloads the results.
      4. Generates a multi-point-to-multi-point (mp2mp) matrix from the results.
      5. Displays the matrix and saves it as a CSV file.
      6. Logs progress and messages in the output widget.

    @param b: Event object passed by `ipywidgets.Button.on_click` (not used directly,
        but required by the callback signature).
    """
    out_status.clear_output()

    if df_tx is None or df_rx is None:
        with out_status:
            print("Both files must be loaded correctly")
        return
    
    if dropdown_model.value is None:
        with out_status:
            print("A model must be selected")
        return
    
    if dropdown_antenna.value is None:
        with out_status:
            print("An antenna must be selected")
        return
    
    df_tx["transmitter id"] = range(1, len(df_tx) + 1)
    df_network = df_tx.merge(df_rx, how="cross")

    with out_status:
        print("Starting calculations")
    create_session()
    create_simulation(df_network, dropdown_antenna.value, dropdown_model.value)
    wait_for_simulation()
    results = download_simulation_results()
    df_matrix = generate_mp2mp_matrix(results)
    with out_status:
        print("Results matrix :")
        display(df_matrix)
        matrix_path = "matrix_received_power.csv"
        df_matrix.reset_index().to_csv(matrix_path, sep=";", index=False)
        print(f"Save results in csv format under {matrix_path}")

btn_run.on_click(on_run_clicked)


## UI Display

In [ ]:
display(ui)